# Plot crashes in Hudson County, 2017-2021 

In [ ]:
from utz import *
from nj_crashes.crashes import Crashes
import plotly.express as px
import plotly.graph_objects as go

## Mapbox configs

In [ ]:
token = open(".mapbox-token").read()
mapbox = dict(mapbox=dict(
    style="dark",
    accesstoken=token,
))

In [ ]:
first_year = 2017
last_year = 2021

In [ ]:
import duckdb
dql = duckdb.sql

In [ ]:
from njdot import NJDOT_DIR
from nj_crashes.paths import PUBLIC_DIR

In [ ]:
%%time
dql(f"IMPORT DATABASE '{NJDOT_DIR}/crashes.ddb'")

In [ ]:
%%time
hpif = dql("""
SELECT id, dt, sri, mp, ilat, ilon, severity
FROM crashes
WHERE icn='Hudson'
AND date_part('year', dt) >= 2017
""").df().set_index('id')

hpif.to_parquet(f'{PUBLIC_DIR}/njdot/hudson-2017:-pif.parquet')
hpif

In [ ]:
%%time
hif = dql("""
SELECT id, dt, sri, mp, ilat, ilon, severity
FROM crashes
WHERE icn='Hudson'
AND date_part('year', dt) >= 2017
AND severity != 'p'
""").df().set_index('id')

hif.to_parquet(f'{PUBLIC_DIR}/njdot/hudson-2017:-if.parquet')
hif

In [ ]:
%%time
hoif = dql("""
SELECT *
FROM crashes
WHERE (icn='Hudson' OR ocn='Hudson')
AND date_part('year', dt) >= 2017
AND severity != 'p'
""").df().set_index('id')

hoif['oilat'] = hoif.olat.fillna(hoif.ilat)
hoif['oilon'] = hoif.olon.fillna(hoif.ilon)
hoif['oicn'] = hoif.ocn.fillna(hoif.icn)
hoif = hoif[hoif.oicn == 'Hudson']
hoif

In [ ]:
hoif.oicn.value_counts()

In [ ]:
diff = hoif[(~hoif.olat.isna()) & (~hoif.ilat.isna()) & ((hoif.olat != hoif.ilat) | (hoif.ilon != hoif.olon))]
diff

## Export crashes

### Reduce + export JSON

In [ ]:
from nj_crashes.json import reduce_df
from nj_crashes.paths import PLOTS_DIR

In [ ]:
outdir = f'{PLOTS_DIR}/njdot'

def reduce_and_save(df, name=None, renames=None, cols=None):
    path = f'{outdir}/{name}' if name else None
    if renames:
        df = df.rename(columns=renames)
    return reduce_df(df, cols=cols, path=path)

In [ ]:
renames = {
    'Total Killed': 'tk',
    'Total Injured': 'ti',
    'Pedestrians Killed': 'pk',
    'Pedestrians Injured': 'pi',
    'Total Vehicles Involved': 'tv',
    'oilat': 'lat',
    'oilon': 'lon',
    'cn': '_cn',
    'oicn': 'cn',
}
cols = {
    'dt': dict(dt_unit='m'),
    'mn': True,
    'tk': False,
    'ti': False,
    'pk': False,
    'pi': False,
    'severity': True,
    'tv': False,
    'sri': True,
    'mp': False,
    'lon': True,
    'lat': True,
}
fi, obj = reduce_and_save(
    hoif,
    name=f'hudson-5yr-lls-if.json',
    renames=renames,
    cols=cols,
)

In [ ]:
obj['rows'][:10]

Cols:

In [ ]:
obj['cols']

Dictionaries:

In [ ]:
obj['dicts']

In [ ]:
%%time
hopif = dql("""
SELECT *
FROM crashes
WHERE icn='Hudson'
AND date_part('year', dt) >= 2017
""").df().set_index('id')

hopif['oilat'] = hopif.olat.fillna(hopif.ilat)
hopif['oilon'] = hopif.olon.fillna(hopif.ilon)
hopif['oicn'] = hopif.ocn.fillna(hopif.icn)
hopif = hopif[hopif.oicn == 'Hudson']
hopif

In [ ]:
pif, pif_obj = reduce_and_save(
    hopif,
    name=f'hudson-5yr-lls-pif.json',
    renames=renames,
    cols=cols,
)
pif

Reduced rows:

In [ ]:
rd, do = reduce_and_save(
    diff,
    name=f'hudson-5yr-lls-if-diffs.json',
    cols={ 
        **{ k: v for k, v in cols.items() if k not in [ 'lat', 'lon' ] },
        'oilat': True, 'oilon': True,
        'ilat': True, 'ilon': True,
    },
    renames={ k: v for k, v in renames.items() if k not in [ 'oilat', 'oilon' ] },
)
rd

## Plot crashes

In [ ]:
hp = hopif.rename(columns={ 'oilat': 'lat', 'oilon': 'lon', })
hp

Check for crashes labeled as happening outside Hudson County:

In [ ]:
hp.cn.value_counts()

In [ ]:
hp.mn.value_counts()

### Compute bounding box, center

In [ ]:
from nj_crashes.geo import get_county_geometries

In [ ]:
cgs = get_county_geometries()
cgs

In [ ]:
bounds = cgs.loc[['Hudson']].bounds
bounds

In [ ]:
bb = bounds.values[0]
min_lon, min_lat, max_lon, max_lat = bb
bb = box(*bb)
centroid = bb.centroid
mid_lon, mid_lat = centroid.x, centroid.y

min_lon, min_lat, max_lon, max_lat

### Compute lat/lng cluster sizes

In [ ]:
from nj_crashes.crashes import LLCrashes

In [ ]:
ll_hist = LLCrashes(hp).ll_hist
ll_hist

In [ ]:
ll_hist[['severity', 'lls_count']].value_counts(sort=False)

### Plot helpers

In [ ]:
from IPython.display import display, Image

In [ ]:
severities = list(CrashSeverity.ch2Name.keys())
severities

In [ ]:
year_str = f"{first_year}" if first_year == last_year else f"{first_year}-{last_year}"
year_str

Plot crashes, with "density" / shadow, and optional exported/inline display formats

In [ ]:
def crash_plot(
    lls,
    county,
    types='pif',
    center=None, zoom=11.4,
    lg='tr', lm=0.02,
    colors=None, scale_radii=None, size='radius',
    marker=None,
    legend=None,
    W=1000, H=1000,
    layers=None,
    cluster=None,
    density=None,
    density_radius=20,
    zmaxs=None,
    write=True,
    map='carto-darkmatter',  # 'dark' will load .mapbox-token
    show=False,
    print_stats=False,
    **kwargs
):
    name = f'{outdir}/{county.lower()}-{types}-crashes'
    scale_radii = scale_radii or [1, 2, 4]
    colors = colors or ['yellow', 'orange', 'red']
    zmaxs = zmaxs or [20, 10, 1]
    center = center or dict(lon=mid_lon, lat=mid_lat)
    if isinstance(center, list):
        center = dict(lon=center[0], lat=center[1])
    severity_dfs = []
    density_traces = []
    for idx, (severity, color, scale_radius, zmax) in enumerate(zip(CrashSeverity.ch2Name.keys(), colors, scale_radii, zmaxs)):
        if severity not in types:
            continue
        severity_df = lls[lls.severity == severity]
        severity_df = severity_df.assign(radius=severity_df.radius * scale_radius)
        severity_dfs.append(severity_df)

        if density:
            density_trace = go.Densitymapbox(
                name=f'{severity} density',
                lon=severity_df.lon, lat=severity_df.lat,
                zmin=0, zmax=zmax,
                radius=density_radius,
                colorscale=[[0, 'black'], [1, color]],
                hoverinfo='none',
                showscale=False,
                legendrank=990+idx,
            )
            density_traces.append(density_trace)
    df = pd.concat(severity_dfs)
    if print_stats:
        num_crashes_by_type = df.severity.value_counts()
        num_crashes = num_crashes_by_type.sum()
        print(f"{num_crashes} crashes:")
        print(str(num_crashes_by_type))
    fig = px.scatter_mapbox(
        df,
        lon='lon', lat='lat',
        size=size,
        #size_max=10,
        color='severity',
        color_discrete_map=dict(zip(severities, colors)),
        hover_data=['dt', 'Crash Location', 'sri', 'mp', 'lls_count',],
        center=center,
        zoom=zoom,
        height=H,
        **kwargs
    )
    legend_bgcolor = '50'
    mapbox_kwargs = dict(style=map)
    if map == 'dark':
        token = open(".mapbox-token").read()
        mapbox_kwargs['accesstoken'] = token
    if density_traces:
        for trace in density_traces:
            fig.add_trace(trace)
        fig.data = fig.data[-len(density_traces):] + fig.data[:len(density_traces)]
    if lg == 'tr':
        legend_pos = dict(yanchor="top", xanchor="right", x=1-lm, y=1-lm)
    elif lg == 'br':
        legend_pos = dict(yanchor="bottom", xanchor="right", x=1-lm, y=lm)
    else:
        raise ValueError(f'Unrecognized legend position: {lg}')
    legend_obj = dict(
        title=dict(text=''),
        **legend_pos,
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    )
    if legend:
        legend_obj.update(legend)
    legend = legend_obj
    label = ' KSI' if types == 'if' else ' fatal' if types == 'f' else ''
    fig.update_layout(
        mapbox=dict(
            # dict(style=<style>) or dict(style="dark", accesstoken=token)
            **mapbox_kwargs,
            layers=layers,
            #layers=[ { "sourcetype": "image", "source": img, "coordinates": coordinates }, ]
        ),
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        title=dict(
            text=f"{county} County{label} crashes ({year_str})",
            x=0.5, y=0.98,
            xanchor='center', yanchor='top',
            font=dict(size=32, color="white")
        ),
        legend=legend,
    )
    scatter_kwargs = dict(
        cluster=cluster,
        #marker=marker if marker else None,
    )
    for idx, (severity, color) in enumerate(zip(severities, colors)):
        fig.update_traces(selector=dict(name=severity), legendrank=1001 + idx, marker=dict(color=color), **scatter_kwargs)
    if cluster:
        fig.update_traces(cluster=dict(enabled=True))
    if write is True or write == 'png':
        path = f'{name}.png'
        fig.write_image(path, width=W, height=H)
        err(f"Wrote {path}")
    if write is True or write == 'json':
        path = f'{name}.json'
        fig.write_json(path)
        err(f"Wrote {path}")
    if show == 'img':
        return Image(fig.to_image(width=W, height=H))
    elif show:
        fig.show(config=dict(displayModeBar=False))
    return fig

In [ ]:
args = (ll_hist.df, 'Hudson',)
show = 'img'
# show = 
def plot_mix(types='pif', **kwargs):
    kw = dict(
        size_max=15,
        scale_radii=[2, 3, 6],
        types=types,
        density=True,
        zoom=11.5,
        lg='br',
        H=1100,
        show=show,
    )
    kw.update(**kwargs)
    crash_plot(*args, **kwargs, map='dark', write='png', print_stats=True, **kw)
    return crash_plot(*args, **kwargs, write='json', **kw)

### Plot crashes by type

In [ ]:
plot_mix('pif')

In [ ]:
plot_mix('if')

In [ ]:
plot_mix('f')

Simple injury+fatal plot, without "density":

In [ ]:
crash_plot(ll_hist, 'Hudson', types='if', size='radius', size_max=10, show='img')